In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Part 1: Import the Housing data and do feature transformations

In [4]:
df= pd.read_csv('https://raw.githubusercontent.com/ishankarve/NeuralNetworks/main/house_price_full_keras.csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [5]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [6]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [7]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](https://github.com/ishankarve/NeuralNetworks/blob/main/multiple_neurons.png?raw=1)

In [8]:
from tensorflow import keras

In [9]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(X.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [10]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1) 
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [11]:
model.fit(X,Y.values,epochs=10,batch_size=32)

Epoch 1/10
16/16 [==============================] - 1s 1ms/step - loss: 101.2854
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 26.8125
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 6.4182
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 1.8715
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 0.9061
Epoch 6/10
16/16 [==============================] - 0s 2ms/step - loss: 0.6534
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 0.5486
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 0.4811
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 0.4289
Epoch 10/10
16/16 [==============================] - 0s 2ms/step - loss: 0.3878


In [12]:
model.predict(X)[:,0]

array([13.225859 , 12.667591 , 13.197348 , 13.193745 , 13.113943 ,
       12.892164 , 12.895981 , 13.042366 , 13.170589 , 13.151875 ,
       13.208368 , 13.022486 , 13.176129 , 13.236931 , 12.734411 ,
       13.232133 , 13.215141 , 13.004396 , 13.224489 , 13.233014 ,
       13.181036 , 13.158105 , 13.2064   , 13.040478 , 13.144719 ,
       13.189043 , 13.174477 , 13.09004  , 12.742765 , 13.160955 ,
       13.148301 , 13.028177 , 13.124872 , 13.031968 , 12.954526 ,
       13.122307 , 12.891342 , 13.201408 , 13.195809 , 13.172816 ,
       13.2146635, 13.155437 , 12.891861 , 13.205409 , 12.895028 ,
       13.2338915, 13.195294 , 13.197348 , 13.229019 , 12.98527  ,
       13.042366 , 13.1427   , 13.154578 , 13.203916 , 12.80052  ,
       13.11577  , 13.090965 , 12.666712 , 13.226767 , 12.897406 ,
       12.98527  , 13.156328 , 13.040478 , 13.164386 , 13.022486 ,
       12.892362 , 13.211294 , 13.240348 , 12.896798 , 13.183191 ,
       13.059299 , 13.146302 , 13.082626 , 13.198877 , 13.2263